<a href="https://colab.research.google.com/github/HarikrishnanK9/DataSymphony/blob/main/Seq2Seq_Simple__Exampleipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 1: Data Preprocessing


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def data_preprocessor(source_sentences, target_sentences):

    source_tokenizer = Tokenizer()
    source_tokenizer.fit_on_texts(source_sentences)
    source_sequences = source_tokenizer.texts_to_sequences(source_sentences)
    source_padded = pad_sequences(source_sequences, padding='post')

    target_tokenizer = Tokenizer()
    target_tokenizer.fit_on_texts(target_sentences)
    target_sequences = target_tokenizer.texts_to_sequences(target_sentences)
    target_padded = pad_sequences(target_sequences, padding='post')

    return source_padded, target_padded, source_tokenizer, target_tokenizer

english_sentences = ['hello', 'world', 'how are you', 'I am fine', 'have a good day']
spanish_sentences = ['hola', 'mundo', 'cómo estás', 'estoy bien', 'ten un buen día']
input_texts, target_texts, source_tokenizer, target_tokenizer = data_preprocessor(english_sentences, spanish_sentences)


#Step 2: Building the Model


In [8]:
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate
from tensorflow.keras.layers import AdditiveAttention as Attention
from tensorflow.keras.models import Model

# Model parameters
embedding_dim = 256
latent_dim = 512
num_encoder_tokens = len(source_tokenizer.word_index) + 1
num_decoder_tokens = len(target_tokenizer.word_index) + 1

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Attention Layer
attention = Attention()
attention_output = attention([decoder_outputs, encoder_outputs])

# Concatenating attention output and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attention_output])

# Dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#Step 3: Training the Model


In [9]:
from tensorflow.keras.utils import to_categorical
decoder_target_data = to_categorical(target_texts, num_decoder_tokens)
model.fit([input_texts, target_texts], decoder_target_data, batch_size=64, epochs=50, validation_split=0.2)

Epoch 1/50
1/1 [==============================] - 4s 4s/step - loss: 2.4009 - accuracy: 0.0000e+00 - val_loss: 2.4058 - val_accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 39ms/step - loss: 2.2543 - accuracy: 0.6250 - val_loss: 2.4170 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 40ms/step - loss: 2.1026 - accuracy: 0.6250 - val_loss: 2.4336 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 65ms/step - loss: 1.8928 - accuracy: 0.6250 - val_loss: 2.4614 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 38ms/step - loss: 1.5963 - accuracy: 0.6250 - val_loss: 2.5131 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 41ms/step - loss: 1.2993 - accuracy: 0.6250 - val_loss: 2.5809 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 40ms/step - loss: 1.2045 - accuracy: 0.6250 - val_loss: 2.6110 - val_accuracy: 0.0000e+

#Step 4: Inference Model


In [10]:
# Encoder Inference Model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder Inference Model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

#Step 5: Translation Function


In [11]:
def translate(input_text):
    # Tokenize and pad the input sequence
    input_seq = source_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=input_texts.shape[1], padding='post')

    # Get the encoder states
    states_value = encoder_model.predict(input_seq)

    # Generate an empty target sequence of length 1
    target_seq = np.zeros((1, 1))

    # Populate the first character of the target sequence with the start character
    target_seq[0, 0] = target_tokenizer.word_index['start']  # Assuming 'start' is the start token

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_tokenizer.index_word[sampled_token_index]
        decoded_sentence += ' ' + sampled_char

        # Exit condition: either hit max length or find stop token.
        if (sampled_char == 'end' or len(decoded_sentence) > 50):  # Assuming 'end' is the end token
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Example usage
translated_sentence = translate("hello")
print(translated_sentence)

1/1 [==============================] - 0s 359ms/step


KeyError: 'start'